<a href="https://colab.research.google.com/github/JSJeong-me/KOSA_Generate_AI_for_Image/blob/main/130-Stable_Diffusion_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="black">Stable Diffusion 시작하는 섹션</font>
output_path = 'digiclau_lab'
ngrok =''

use_gdrive = 'yes'
class GDriveSaveMode:
    ON = 1
    OFF = 0
if use_gdrive == 'yes':
  is_drive_used = True
elif use_gdrive == 'no':
  is_drive_used = False
else:
  raise ValueError("use_gdrive value invalid.")

Deliberate = False
DreamShaper = False
GhostMix = False
RealisticVision = False
MajicMIXRealistic = False
AbyssOrangeMix3 = False
CounterfeitV30 = False
ReVAnimated = False
Lyriel = False
NeverEndingDream = False

SDXL_10_Base_AND_Refiner = True

Model_URL = 'https://civitai.com/api/download/models/138176'

ControlNet = False

Deforum = False

SadTalker = False

Regional_Prompter = False
Ultimate_SD_Upscale = False
Openpose_Editor = False
ADetailer = False

LyCORIS = False

AnimateDiff = False

Text2Video = False
DynamicThresholding = False

DWPose = False

Roop = False

LoopbackWave = False


# 로그 삭제
#Clear_Log = True #@param{type:'boolean'}


# #@markdown ## 🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨
# #@markdown ## 'COMMANDLINE_ARGS=' 에 추가할 값이 있다면 이곳에 넣어주세요.
# Extra_args = '' #@param {type: "string"}


if is_drive_used:
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  # Colab에 설치
  root = '/content/'
  # 구글 드라이브에 필요한 폴터 생성
  !mkdir -p {output_path}/outputs
  !mkdir -p {output_path}/checkpoints
  !mkdir -p {output_path}/ESRGAN
  !mkdir -p {output_path}/hypernetworks
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path # Is it still needed?
  root = '/content/'

def print_message(msg):
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  # print('📍📍📍 ', msg, ' 📍📍📍')
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  pass

def clear():
    from IPython.display import clear_output
    return clear_output()

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  elif 'civitai.com' in url:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}
  else:
    print_message('잘못된 모델 URL입니다.')

def checkpoint_exists(): ### See if a checkpoint model exists in goodle drive ###
  files = os.listdir(output_path + '/checkpoints/') # get all files
  for file in files: # check .ckpt or .safetensors extension
    if file.endswith('.ckpt') or file.endswith('.safetensors'):
      return True
  return False # no checkpoint file

def download_models():
  models_count = 0
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print_message('사용할 모델을 다운받고 있습니다.')
  if Deliberate:
    downloadModel('https://civitai.com/api/download/models/15236')
    models_count += 1
  if DreamShaper:
    downloadModel('https://civitai.com/api/download/models/128713')
    models_count += 1
  if GhostMix:
    downloadModel('https://civitai.com/api/download/models/76907')
    models_count += 1
  if RealisticVision:
    downloadModel('https://civitai.com/api/download/models/130072')
    models_count += 1
  if MajicMIXRealistic:
    downloadModel('https://civitai.com/api/download/models/94640')
    models_count += 1
  if AbyssOrangeMix3:
    downloadModel('https://civitai.com/api/download/models/17233')
    models_count += 1
  if CounterfeitV30:
    downloadModel('https://civitai.com/api/download/models/57618')
    models_count += 1
  if ReVAnimated:
    downloadModel('https://civitai.com/api/download/models/46846')
    models_count += 1
  if Lyriel:
    downloadModel('https://civitai.com/api/download/models/72396')
    models_count += 1
  if NeverEndingDream:
    downloadModel('https://civitai.com/api/download/models/64094')
    models_count += 1
  if SDXL_10_Base_AND_Refiner:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')
    models_count += 1
  if Model_URL:
      for m in Model_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)

  # 사용할 모델이 없으면, SD 1.5모델은 필수로 다운로드
  if models_count == 0 and not checkpoint_exists():
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
    models_count += 1
  print_message(str(models_count) + '개의 Checkpoint 모델을 다운 받았습니다.')


  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def install_extensions():
  # extensions directory
  %cd {root}/stable-diffusion-webui/extensions

  if ControlNet:
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    %cd {root}/stable-diffusion-webui/extensions
  if Deforum:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui
  if SadTalker:
    !git clone https://github.com/OpenTalker/SadTalker
    !mkdir -p {root}/stable-diffusion-webui/models/SadTalker
    %cd {root}/stable-diffusion-webui/models/SadTalker
    url = 'https://huggingface.co/digiclau/files/resolve/main/extensions/SadTalker/sadtalker_checkpoints-20230517T015718Z.zip'
    filename = "sadtalker_checkpoints.zip"
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
    !unzip {filename}
    !rm {filename}
    # extra code to fix error
    !mkdir /tmp
    !mkdir /tmp/gradio
    %cd {root}/stable-diffusion-webui/extensions
  if Regional_Prompter:
    !git clone https://github.com/hako-mikan/sd-webui-regional-prompter
  if Ultimate_SD_Upscale:
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111
  if Openpose_Editor:
    !git clone https://github.com/fkunn1326/openpose-editor
  if ADetailer:
    !git clone https://github.com/Bing-su/adetailer
  if LyCORIS:
    !git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-lycoris
  if AnimateDiff:
    !git clone https://github.com/continue-revolution/sd-webui-animatediff
    %cd {root}/stable-diffusion-webui/extensions/sd-webui-animatediff/model
    downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt')
    downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt')
    %cd {root}/stable-diffusion-webui/extensions
  if Text2Video:
    !git clone https://github.com/kabachuha/sd-webui-text2video
    %mkdir -p {root}/stable-diffusion-webui/models/text2video/t2v
    %cd {root}/stable-diffusion-webui/models/text2video/t2v
    downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/VQGAN_autoencoder.pth')
    downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/configuration.json')
    downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/open_clip_pytorch_model.bin')
    downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/text2video_pytorch_model.pth')
    %cd {root}/stable-diffusion-webui/extensions
  if DynamicThresholding:
    !git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding
  if DWPose:
    !git clone https://github.com/IDEA-Research/DWPose
  if Roop:
    !git clone https://github.com/s0md3v/sd-webui-roop
    filePath = f"{root}/stable-diffusion-webui/extensions/sd-webui-roop/install.py"
    oldDeadFile = 'https://huggingface.co/deepinsight/inswapper/resolve/main/inswapper_128.onnx'
    newLiveFile = 'https://huggingface.co/facefusion/models/resolve/main/inswapper_128.onnx'
    oldDeadFile = oldDeadFile.replace('/', '\/')
    newLiveFile = newLiveFile.replace('/', '\/')
    !sed -i 's/{oldDeadFile}/{newLiveFile}/g' {filePath}

#  if Depthmap_3D_Painting:
#    !git clone https://github.com/thygate/stable-diffusion-webui-depthmap-script

def install_scripts():
  if LoopbackWave:
    %cd {root}/stable-diffusion-webui/scripts
#    url = 'https://files.catbox.moe/0hx51x.py'
#    filename = 'loopback_wave.py'
#    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
    url = 'https://github.com/DigiClau/stablediffusion_webui/raw/main/scripts/loopback_wave.py'
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print_message('이미 python 3.10이 설치되어 있습니다.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/config.json -O {output_path + '/config.json'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all checkpoints in the checkpoints folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  checkpoints_in_google_drive = glob.glob(output_path + '/checkpoints/*')
  print_message('구글 드라이브에 들어 있는 Checkpoint 모델: %s'%checkpoints_in_google_drive)
  for f in checkpoints_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print_message('구글 드라이브에 들어 있는 Upscaler 모델: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use LyCORIS model folder in google drive
  !mkdir -p {output_path}/LyCORIS
  %cd {root}/stable-diffusion-webui/models
  !rm -rf LyCORIS
  !ln -s {output_path}/LyCORIS

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    !git clone {url}

def lowRamPatch():
  !sed -i 's/dict()))$/dict())).cuda()/g'  {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py

updatePython()
!mkdir -p {root}
%cd {root}
# os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# A1111 first launch
%cd {root}/stable-diffusion-webui
# !git checkout -f 68f336bd994bed5442ad95bad6b6ad5564a5409a # 1.5.1
!git checkout -f c9c8485bc1e8720aba70f029d25cba1c4abf2b5c # 1.5.2
!COMMANDLINE_ARGS="--exit"  python launch.py

if is_drive_used: # == GDriveSaveMode.ON:
  initSaveGoogleDriveModelOnly()

In [ ]:
download_models()

In [ ]:
install_extensions()

In [ ]:
install_scripts()

In [ ]:
clear()

In [ ]:
lowRamPatch()

In [ ]:
%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers'

if ngrok:
  args += f' --ngrok {ngrok} '
else:
  args += ' --share '
# args+= ' ' + Extra_args
print(f'WEBUI ARGUMENTS: {args}')


In [ ]:
!python {root}/stable-diffusion-webui/launch.py {args}

In [ ]:
# from modules import launch_utils


# args = launch_utils.args
# python = launch_utils.python
# git = launch_utils.git
# index_url = launch_utils.index_url
# dir_repos = launch_utils.dir_repos

# commit_hash = launch_utils.commit_hash
# git_tag = launch_utils.git_tag

# run = launch_utils.run
# is_installed = launch_utils.is_installed
# repo_dir = launch_utils.repo_dir

# run_pip = launch_utils.run_pip
# check_run_python = launch_utils.check_run_python
# git_clone = launch_utils.git_clone
# git_pull_recursive = launch_utils.git_pull_recursive
# list_extensions = launch_utils.list_extensions
# run_extension_installer = launch_utils.run_extension_installer
# prepare_environment = launch_utils.prepare_environment
# configure_for_tests = launch_utils.configure_for_tests
# start = launch_utils.start


# def main():
#     if not args.skip_prepare_environment:
#         prepare_environment()

#     if args.test_server:
#         configure_for_tests()

#     start()


# if __name__ == "__main__":
#     main()